In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_xy,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [11]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [13]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [41]:
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [42]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [43]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [44]:
len(df_ipcc_group)

53258

In [45]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

True

In [46]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [47]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [48]:
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [49]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [50]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [51]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

True

In [115]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')

In [116]:
df_ipcc_ipbes['countries']=df_ipcc_ipbes.apply(lambda row: get_xy('countries',row), axis=1)
df_ipcc_ipbes['year']=df_ipcc_ipbes.apply(lambda row: get_xy('year',row), axis=1)
df_ipcc_ipbes['title']=df_ipcc_ipbes.apply(lambda row: get_xy('title',row), axis=1)
df_ipcc_ipbes['rors']=df_ipcc_ipbes.apply(lambda row: get_xy('rors',row), axis=1)
df_ipcc_ipbes['authors_name']=df_ipcc_ipbes.apply(lambda row: get_xy('authors_name',row), axis=1)
df_ipcc_ipbes['institutions_names']=df_ipcc_ipbes.apply(lambda row: get_xy('institutions_names',row), axis=1)

In [117]:
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year','title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes']]

In [118]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

True

In [ ]:
update_bso_publications()

In [29]:
bso=pd.DataFrame(get_bso_publications())

In [30]:
df_bso=bso.transpose().reset_index()

In [31]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [82]:
#pour faire une liste de rors
#df_ipcc_ipbes.loc[:,'rors']=df_ipcc_ipbes.loc[:,'rors'].apply(lambda x: aplatir([list(y.keys()) for y in aplatir(x)]) if isinstance(x,list) else None)
#df_ipcc_ipbes.loc[:,'rors']=df_ipcc_ipbes.loc[:,'rors'].apply(lambda x: [z.replace('https://ror.org/','') for z in  x] if isinstance(x,list) else None)                          

In [119]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [120]:
del df_ipcc_ipbes_bso['rors_y']

In [121]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [122]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes', 'bso_local_affiliations'],
      dtype='object')

In [91]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [92]:
#df_iso=df_iso.transpose().reset_index().rename(columns={'index':'code',0:'name'})

In [123]:
dict_iso={}
for i,row in df_iso.iterrows():
    dict_iso[row['code']]=row['name']
dict_iso['CW']='Curaçao'
dict_iso['RS']='Serbia'
dict_iso['ME']='Montenegro'
dict_iso['XK']='Kosovo'
dict_iso['SS']='South Soudan'
dict_iso['MF']='Saint Martin'

In [124]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [125]:
df_brevet_doi=pd.read_json('brevet_doi.json')

In [96]:
df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [97]:
len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

False

In [ ]:
df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

In [127]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54821 data in cached openalex data
6608 data in cached openalex data


In [128]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [129]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [135]:
dict_doi_bso=[]

In [136]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [137]:
len(dict_doi_bso)

58240

In [138]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [139]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [2]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', lines=True)

In [3]:
len(dd)

58240

In [4]:
current_dict = dd.to_dict(orient='records')

In [5]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [6]:
df_elastic_input=pd.DataFrame(current_dict)

In [7]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [8]:
df_elastic_input=df_elastic_input[['doi', 'year', 'title', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes','openalex_topics']]

In [9]:
df_elastic_input.loc[:,'openalex_topics']=df_elastic_input.loc[:,'openalex_topics'].apply(lambda x: [y.get('display_name') for y in x] if isinstance(x,list) else [])

In [22]:
df_elastic_input.institutions_names[600]

[[{'GEOMAR Helmholtz Centre for Ocean Research Kiel': 'DE'}],
 [{'GEOMAR Helmholtz Centre for Ocean Research Kiel': 'DE'},
  {'University of Graz': 'AT'}],
 [{'GEOMAR Helmholtz Centre for Ocean Research Kiel': 'DE'},
  {'A.M. Obukhov Institute of Atmospheric Physics': 'RU'},
  {'Institute of Oceanology. PP Shirshov Russian Academy of Sciences': 'RU'},
  {'Institute of Geography': 'RU'}],
 [{'GEOMAR Helmholtz Centre for Ocean Research Kiel': 'DE'}]]

In [23]:
df_elastic_input.institutions_names[1289]

[[{'A.M. Obukhov Institute of Atmospheric Physics': 'RU'}],
 [{'Alfred-Wegener-Institut Helmholtz-Zentrum für Polar- und Meeresforschung': 'DE'}],
 [{'A.M. Obukhov Institute of Atmospheric Physics': 'RU'},
  {'Lomonosov Moscow State University': 'RU'}],
 [{'Alfred-Wegener-Institut Helmholtz-Zentrum für Polar- und Meeresforschung': 'DE'}],
 [{'A.M. Obukhov Institute of Atmospheric Physics': 'RU'},
  {'Institute of Geography': 'RU'}],
 [{'Bjerknes Centre for Climate Research': 'NO'},
  {'Uni Research (Norway)': 'NO'}],
 [{'Cooperative Institute for Research in Environmental Sciences': 'US'},
  {'University of Colorado Boulder': 'US'}],
 [{'Bjerknes Centre for Climate Research': 'NO'},
  {'Uni Research (Norway)': 'NO'},
  {'University of Copenhagen': 'DK'}],
 [{'A.M. Obukhov Institute of Atmospheric Physics': 'RU'}],
 [{'Alfred-Wegener-Institut Helmholtz-Zentrum für Polar- und Meeresforschung': 'DE'}],
 [{'University of Liège': 'BE'}],
 [{'Iowa State University': 'US'}],
 [{'Max Planck In

In [15]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [16]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

fichier excel

In [63]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [64]:
df.loc[:,'rors']=df.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'authors_name']=df.loc[:,'authors_name'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'institutions_names']=df.loc[:,'institutions_names'].apply(lambda x: [[]] if x==[None] else x)

In [65]:
df.loc[pd.isna(df['rors'])==False,'rors']=df.loc[pd.isna(df['rors'])==False,'rors'].apply(lambda x: ','.join([j.replace('https://ror.org/','') for j in list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())]))

In [66]:
df.loc[pd.isna(df['institutions_names'])==False,'institutions_names']=df.loc[pd.isna(df['institutions_names'])==False,'institutions_names'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())))

In [67]:
df.loc[pd.isna(df['authors_name'])==False,'authors_name']=df.loc[pd.isna(df['authors_name'])==False,'authors_name'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y.keys()) for y in x if isinstance(y,dict)])).drop_duplicates())))

In [70]:
df.loc[pd.isna(df['ipcc'])==False,'ipcc']=df.loc[pd.isna(df['ipcc'])==False,'ipcc'].apply(lambda x: ','.join(list((pd.Series([y.get('name') for y in x]).drop_duplicates()))))

In [71]:
df.loc[pd.isna(df['ipbes'])==False,'ipbes']=df.loc[pd.isna(df['ipbes'])==False,'ipbes'].apply(lambda x: ','.join(list((pd.Series([y.get('chapter') for y in x]).drop_duplicates()))))

In [ ]:
df['is_fr']=df['countries'].apply(lambda x: 'France' in x if isinstance(x,list) else False)

In [83]:
df.loc[pd.isna(df['countries'])==False,'countries']=df.loc[pd.isna(df['countries'])==False,'countries'].apply(lambda x: ','.join(x))

In [85]:
df[['doi','title','ipcc','ipbes','countries','is_fr','rors','institutions_names','authors_name']].to_excel('doi_ipcc_ipbes.xlsx', index=False)